# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 6. Support Vector Machine (SVM)


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [ ]:
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-04-29 17:08:36--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.7’

iris.csv.7          100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-04-29 17:08:36 (54.7 MB/s) - ‘iris.csv.7’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('iris.csv')




Cek isi dataset Anda dengan menggunakan perintah **head()**

In [ ]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [ ]:
uniq = data['variety'].unique()
print(uniq)

['Setosa' 'Versicolor' 'Virginica']


Metode Support Vector Machine (SVM) dasar hanya mampu melakukan klasifikasi data yang terdiri dari dua kelas, atau disebut klasifikasi biner. Dataset iris memiliki 3 kelas, sehingga salah satu kelasnya, yaitu Iris-virginica perlu dihapus. Cell berikut menghapus data yang memiliki kelas Iris-virginica

In [ ]:
praktikum_data = data.copy()

In [ ]:
praktikum_data.drop(praktikum_data[praktikum_data['variety']=='Virginica'].index, inplace = True)

SVM juga memiliki keterbatasan, yaitu hanya mampu menerima kelas dalam bentuk numerik. Cell berikut mengubah kelas menjadi bilangan, Iris setosa menjadi -1 dan Iris-versicolor menjadi 1

In [ ]:
praktikum_data['variety'] = praktikum_data['variety'].map({'Setosa':-1, 'Versicolor':1})

In [ ]:
praktikum_data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,-1
1,4.9,3.0,1.4,0.2,-1
2,4.7,3.2,1.3,0.2,-1
3,4.6,3.1,1.5,0.2,-1
4,5.0,3.6,1.4,0.2,-1


In [ ]:
praktikum_data['variety'].unique()

array([-1,  1])

## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [ ]:
from sklearn.model_selection import train_test_split

data_latih, data_uji = train_test_split(praktikum_data, test_size = 0.2, random_state=42)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 80 data, dan **data_uji** terdiri dari 20 data

In [ ]:
print(data_latih.shape[0])
print(data_uji.shape[0])

80
20


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_latih** dan **label_uji**

In [ ]:
label_latih = data_latih.pop('variety')
label_uji = data_uji.pop('variety')

In [ ]:
label_latih.info()

<class 'pandas.core.series.Series'>
Index: 80 entries, 55 to 51
Series name: variety
Non-Null Count  Dtype
--------------  -----
80 non-null     int64
dtypes: int64(1)
memory usage: 1.2 KB


## 3) Proses Training

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [ ]:
def hitung_cost_gradient(W,X,Y, regularization):
  jarak = 1-(Y*np.dot(X,W))
  dw=np.zeros(len(W))
  if max(0, jarak) == 0:
    di=W
  else:
    di= W-(regularization*Y*X)
  dw += di
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

In [ ]:
from sklearn.utils import shuffle

def sgd(data_latih, label_latih, learning_rate = 0.000001, max_epoch = 1000, regularization = 10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1, max_epoch):
    X,Y = shuffle(data_latih, label_latih, random_state=101)
    for index, x in enumerate (X):
      delta = hitung_cost_gradient(bobot, x ,Y[index], regularization)
      bobot = bobot - (learning_rate*delta)
    return bobot

Proses training dilakukan dengan memanggil fungsi **sgd** dengan parameter input berupa data latih dan label latih. Parameter learning rate dan max epoch menggunakan nilai *default* nya

In [ ]:
W = sgd(data_latih, label_latih)
print(W)

[-0.08599658 -0.33298696  0.50598027  0.21799146]


## 4) Proses *testing*
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [ ]:
def testing(W, data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W, data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi, y_prediksi)
  return prediksi

Lakukan pengujian menggunakan seluruh data uji. Bandingkan hasilnya dengan nilai label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar.

In [ ]:
y_prediksi = testing(W, data_uji)
print(sum(y_prediksi==label_uji))

20


## TUGAS
Pada tugas kali ini Anda masih melakukan klasifikasi dataset iris, tetapi menggunakan kelas Iris-versicolor dan Iris-virginica. Apakah hasilnya lebih baik jika dibandingkan dengan Iris-setosa dan Iris-versicolor?

In [ ]:
tugas_data = data.copy()

In [ ]:
tugas_data['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [ ]:
tugas_data.drop(tugas_data[tugas_data['variety']=='Setosa'].index, inplace = True)

In [ ]:
tugas_data['variety'] = tugas_data['variety'].map({'Versicolor':1, 'Virginica':-1})

In [ ]:
tugas_data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1


In [ ]:
tugas_data['variety'].unique()

array([ 1, -1])

In [ ]:
tugas_data_latih, tugas_data_uji = train_test_split(tugas_data, test_size = 0.2)

In [ ]:
tugas_data_latih.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 76 to 59
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  80 non-null     float64
 1   sepal.width   80 non-null     float64
 2   petal.length  80 non-null     float64
 3   petal.width   80 non-null     float64
 4   variety       80 non-null     int64  
dtypes: float64(4), int64(1)
memory usage: 3.8 KB


In [ ]:
print(tugas_data_latih.shape[0])
print(tugas_data_uji.shape[0])

80
20


In [ ]:
tugas_label_latih = tugas_data_latih.pop('variety')
tugas_label_uji = tugas_data_uji.pop('variety')

In [ ]:
tugas_label_latih.info()

<class 'pandas.core.series.Series'>
Index: 80 entries, 76 to 59
Series name: variety
Non-Null Count  Dtype
--------------  -----
80 non-null     int64
dtypes: int64(1)
memory usage: 1.2 KB


In [ ]:
tugas_W = sgd(tugas_data_latih, tugas_label_latih)
print(tugas_W)

[ 0.17199811  0.13899748 -0.0679936  -0.08699521]


In [ ]:
tugas_y_prediksi = testing(tugas_W, tugas_data_uji)
print(sum(tugas_y_prediksi==tugas_label_uji))

12


Hasilnya tidak lebih baik karena hanya mampu mengenali 12 diantara 20 data uji.

Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi **sgd**. Apakah hasilnya lebih baik? Mengapa?

In [ ]:
from sklearn.utils import shuffle

def tugas_sgd(data_latih, label_latih, learning_rate = 0.000001, max_epoch = 5000, regularization = 10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1, max_epoch):
    X,Y = shuffle(data_latih, label_latih, random_state=101)
    for index, x in enumerate (X):
      delta = hitung_cost_gradient(bobot, x ,Y[index], regularization)
      bobot = bobot - (learning_rate*delta)
    return bobot

In [ ]:
tugas_W_2 = tugas_sgd(tugas_data_latih, tugas_label_latih)
print(tugas_W_2)

[ 0.17199811  0.13899748 -0.0679936  -0.08699521]


In [ ]:
tugas_y_prediksi_2 = testing(tugas_W_2, tugas_data_uji)
print(sum(tugas_y_prediksi_2==tugas_label_uji))

12


Tidak, jumlah max-epoch dalam fungsi di atas merupakan jumlah iterasi. Pada contoh di atas, ketika jumlah iterasi adalah 1000, malah sama  meskipun diubah ke 5000. Kemungkinan disebabkan oleh model yang telah mencapai nilai optimal sebelum mencapai 5000 iterasi atau bahkan pada iterasi ke-1000. Iterasi tambahan hanya akan menjauhkan model dari titik optimalnya.

Terakhir, lakukanlah seleksi fitur sederahana dengan mengambil hanya fitur sepal length dan sepal width saja. Apakah hasilnya lebih baik?

In [ ]:
tugas_data_2 = tugas_data.loc[:, ['sepal.length', 'sepal.width', 'variety']]

In [ ]:
tugas_data_2.head()

,sepal.length,sepal.width,variety
50,7.0,3.2,1
51,6.4,3.2,1
52,6.9,3.1,1
53,5.5,2.3,1
54,6.5,2.8,1


In [ ]:
tugas_data_latih_2, tugas_data_uji_2 = train_test_split(tugas_data_2, test_size=0.2)

In [ ]:
tugas_label_latih_2 = tugas_data_latih_2.pop('variety')
tugas_label_uji_2 = tugas_data_uji_2.pop('variety')

In [ ]:
tugas_label_latih_2.info()

<class 'pandas.core.series.Series'>
Index: 80 entries, 87 to 110
Series name: variety
Non-Null Count  Dtype
--------------  -----
80 non-null     int64
dtypes: int64(1)
memory usage: 1.2 KB


In [ ]:
tugas_W_3 = tugas_sgd(tugas_data_latih_2, tugas_label_latih_2)
print(tugas_W_3)

[-0.19299774 -0.06999966]


In [ ]:
tugas_y_prediksi_3 = testing(tugas_W_3, tugas_data_uji_2)
print(sum(tugas_y_prediksi_3==tugas_label_uji_2))

9


Tidak. Penggunaan 2 fitur lebih buruk, karena penggunaan 4 fitur lebih baik daripada hanya menggunakan 2 fitur dalam klasifikasi. SVM juga lebih unggul dalam klasifikasi data dengan dimensi tinggi. Namun, ketika dimensi dikurangi, kemampuan model untuk memahami dan membedakan pola yang rumit dalam data dapat berkurang.